In [3]:
import argparse

import pandas as pd
import torch
from thop import profile, clever_format
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import Model, set_bn_eval
from utils import recall, LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler

dev = torch.device("cuda:0")  

def train(net, optim):
    torch.cuda.empty_cache()
    net.train()
    # fix bn on backbone network
    net.apply(set_bn_eval)
    total_loss, total_correct, total_num, data_bar = 0, 0, 0, tqdm(train_data_loader)
    for i, (inputs, labels) in enumerate(data_bar):
        #if i > 5000:
        #    break
        inputs, labels = inputs.cuda(), labels.cuda()
        features, classes = net(inputs)
        class_loss = class_criterion(classes, labels)
        feature_loss = feature_criterion(features, labels)
        loss = class_loss + feature_loss
        optim.zero_grad()
        loss.backward()
        optim.step()
        pred = torch.argmax(classes, dim=-1)
        total_loss += loss.item() * inputs.size(0)
        total_correct += torch.sum(pred == labels).item()
        total_num += inputs.size(0)
        data_bar.set_description('Train Epoch {}/{} - Loss:{:.4f} - Acc:{:.2f}%'
                                 .format(epoch, num_epochs, total_loss / total_num, total_correct / total_num * 100))

    return total_loss / total_num, total_correct / total_num * 100


def test(net, recall_ids):
    net.eval()
    with torch.no_grad():
        # obtain feature vectors for all data
        for key in eval_dict.keys():
            eval_dict[key]['features'] = []
            for inputs, labels in tqdm(eval_dict[key]['data_loader'], desc='processing {} data'.format(key)):
                inputs, labels = inputs.cuda(), labels.cuda()
                features, classes = net(inputs)
                eval_dict[key]['features'].append(features)
            eval_dict[key]['features'] = torch.cat(eval_dict[key]['features'], dim=0)

        # compute recall metric
        if data_name == 'isc':
            acc_list = recall(eval_dict['test']['features'], test_data_set.labels, recall_ids,
                              eval_dict['gallery']['features'], gallery_data_set.labels)
        else:
            acc_list = recall(eval_dict['test']['features'], test_data_set.labels, recall_ids)
    desc = 'Test Epoch {}/{} '.format(epoch, num_epochs)
    for index, rank_id in enumerate(recall_ids):
        desc += 'R@{}:{:.2f}% '.format(rank_id, acc_list[index] * 100)
        results['test_recall@{}'.format(rank_id)].append(acc_list[index] * 100)
    print(desc)
    return acc_list[0]


'''
parser = argparse.ArgumentParser(description='Train CGD')
parser.add_argument('--data_path', default='/home/data', type=str, help='datasets path')
parser.add_argument('--data_name', default='car', type=str, choices=['car', 'cub', 'sop', 'isc'],
                    help='dataset name')
parser.add_argument('--crop_type', default='uncropped', type=str, choices=['uncropped', 'cropped'],
                    help='crop data or not, it only works for car or cub dataset')
parser.add_argument('--backbone_type', default='resnet50', type=str, choices=['resnet50', 'resnext50'],
                    help='backbone network type')
parser.add_argument('--gd_config', default='SG', type=str,
                    choices=['S', 'M', 'G', 'SM', 'MS', 'SG', 'GS', 'MG', 'GM', 'SMG', 'MSG', 'GSM'],
                    help='global descriptors config')
parser.add_argument('--feature_dim', default=1536, type=int, help='feature dim')
parser.add_argument('--smoothing', default=0.1, type=float, help='smoothing value for label smoothing')
parser.add_argument('--temperature', default=0.5, type=float,
                    help='temperature scaling used in softmax cross-entropy loss')
parser.add_argument('--margin', default=0.1, type=float, help='margin of m for triplet loss')
parser.add_argument('--recalls', default='1,2,4,8', type=str, help='selected recall')
parser.add_argument('--batch_size', default=128, type=int, help='train batch size')
parser.add_argument('--num_epochs', default=20, type=int, help='train epoch number')

opt = parser.parse_args()
'''
# args parse
data_path, data_name, crop_type, backbone_type = ('/Users/vepakommac/Documents/PrivateMail/CGD-master/CUB_200_2011/CUB_200_2011', 'cub', 'cropped', 'resnet50')
gd_config = 'G'
feature_dim = 64
smoothing, temperature  = (0.1, 0.5)
margin = 0.1
recalls = [1,2,4,8]
batch_size = 32
num_epochs = 20
save_name_pre = '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(data_name, crop_type, backbone_type, gd_config, feature_dim,
                                                    smoothing, temperature, margin, batch_size)

results = {'train_loss': [], 'train_accuracy': []}
for recall_id in recalls:
    results['test_recall@{}'.format(recall_id)] = []

# dataset loader
train_data_set = ImageReader(data_path, data_name, 'train', crop_type)
train_sample = MPerClassSampler(train_data_set.labels, batch_size)
train_data_loader = DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
test_data_set = ImageReader(data_path, data_name, 'query' if data_name == 'isc' else 'test', crop_type)
test_data_loader = DataLoader(test_data_set, batch_size, shuffle=False, num_workers=8)
eval_dict = {'test': {'data_loader': test_data_loader}}
if data_name == 'isc':
    gallery_data_set = ImageReader(data_path, data_name, 'gallery', crop_type)
    gallery_data_loader = DataLoader(gallery_data_set, batch_size, shuffle=False, num_workers=8)
    eval_dict['gallery'] = {'data_loader': gallery_data_loader}

# model setup, model profile, optimizer config and loss definition
model = Model(backbone_type, gd_config, feature_dim, num_classes=len(train_data_set.class_to_idx)).cuda()
flops, params = profile(model, inputs=(torch.randn(1, 3, 224, 224).cuda(),))
flops, params = clever_format([flops, params])
print('# Model Params: {} FLOPs: {}'.format(params, flops))
optimizer = Adam(model.parameters(), lr=1e-4)
lr_scheduler = MultiStepLR(optimizer, milestones=[int(0.6 * num_epochs), int(0.8 * num_epochs)], gamma=0.1)
class_criterion = LabelSmoothingCrossEntropyLoss(smoothing=smoothing, temperature=temperature)
feature_criterion = BatchHardTripletLoss(margin=margin)

best_recall = 0.0
for epoch in range(1, num_epochs + 1):
    train_loss, train_accuracy = train(model, optimizer)
    results['train_loss'].append(train_loss)
    results['train_accuracy'].append(train_accuracy)
    rank = test(model, recalls)
    lr_scheduler.step()

    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    data_frame.to_csv('results/{}_statistics.csv'.format(save_name_pre), index_label='epoch')
    # save database and model
    data_base = {}
    if rank > best_recall:
        best_recall = rank
        data_base['test_images'] = test_data_set.images
        data_base['test_labels'] = test_data_set.labels
        data_base['test_features'] = eval_dict['test']['features']
        if data_name == 'isc':
            data_base['gallery_images'] = gallery_data_set.images
            data_base['gallery_labels'] = gallery_data_set.labels
            data_base['gallery_features'] = eval_dict['gallery']['features']
        torch.save(model.state_dict(), 'results/{}_model.pth'.format(save_name_pre))
        torch.save(data_base, 'results/{}_data_base.pth'.format(save_name_pre))



Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /Users/vepakommac/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


AssertionError: Torch not compiled with CUDA enabled

In [4]:
pip install thop

Note: you may need to restart the kernel to use updated packages.
